# Pendahuluan

In [1]:
# Library yang dibutuhkan

# Pertama-tama, import terlebih dahulu library yang dibutuhkan, antara lain secara berurutan adalah :
# json,
# numpy,
# pandas, dan
# requests.

# Library numpy dan pandas diimport sebagai aliasnya.

import json
import numpy as np
import pandas as pd
import requests

In [ ]:
# Membuat Fungsi get API

# Buat fungsi python get_json dengan parameter api_url. fungsi ini akan mengembalikan value berupa python dictionary. 
# Jika status_code yang dihasilkan adalah 200. Jika tidak, maka value yang dikembalikan adalah None.